### LCEL 인터페이스

    - 사용자 정의 체인을 가능한 쉽게 만들 수 있도록 Runnable 프로토콜 구현
    Runnable 프로토콜은 대부분의 컴포넌트에 구성되어 있음
    표준 인터페이스이고, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듬

    [표준 인터페이스]
    - stream : 응답의 청크를 스트리밍
    - invoke : 입력에 대한 체인 호출
    - batch :  입력 목록에 대해 체인을 호출

    [비동기 메소드]
    - astream : 비동기적으로 응답의 청크를 스트리밍
    - ainvoke : 비동기적으로 입력에 대한 체인 호출
    - abatch : 비동기적으로 입력 목록에 대해 체인 호출
    - astream_log : 최종 응답뿐만 앙니라 바생하는 중간 단계를 스트리밍

In [3]:
from dotenv import load_dotenv


load_dotenv()

True

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model_name='gpt-3.5-turbo')
prompt = PromptTemplate.from_template("{topic}에 대해 3문장으로 설명해줘.")

chain = prompt | model | StrOutputParser()

In [22]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic}에 대해 3문장으로 설명해줘.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1072a6b40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x107248e60>, root_client=<openai.OpenAI object at 0x103ed8da0>, root_async_client=<openai.AsyncOpenAI object at 0x1072a53a0>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

#### stream : 실시간 출력

    - chain.stream 메서드를 사용해 주어진 토픽에 대한 데이터 스트림 생성
    이 스트림을 반복해 각 데이터 내용(content)를 즉시 출력, end= ""인자는 출력후 줄바꿈을 하지 않도록 설정
    flush=True 인자는 출력 버퍼를 즉시 비움

In [26]:
answer= chain.stream({"topic" : "멀티모달"})

answer

<generator object RunnableSequence.stream at 0x11099ab60>

In [27]:
for token in answer:
    print(token, end="", flush=True)

멀티모달은 여러 가지 다른 형태의 정보를 결합하여 제공하는 시스템이다. 이는 텍스트, 이미지, 음성, 동영상 등 다양한 형태의 정보를 동시에 활용할 수 있다는 것을 의미한다. 멀티모달은 사용자에게 더 풍부한 경험을 제공하고, 정보를 빠르고 효과적으로 전달할 수 있는 장점을 가지고 있다.

#### invoke: 호출

In [28]:
chain.invoke({"topic": "ChatGPT" })

'ChatGPT는 인공지능 챗봇으로, 자연어 처리 기술을 사용하여 대화 상대와 상호작용합니다. 사용자의 질문에 답변하거나 대화를 이끌어내는 등 다양한 상황에서 활용됩니다. 지속적인 학습과 업데이트를 통해 더욱 정확하고 유용한 대화를 제공합니다.'

##### Batch : 배치(단위 실행)
    - chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 topic 키의 키값을 사용해 일괄 처리를 수행

In [30]:
answer = chain.batch([{"topic" : "Chatgpt"}, {"topic" : "Instagram"}])
answer

['ChatGPT는 최신 자연어 처리 기술을 활용하여 대화를 이어나가는 인공지능 챗봇이다. 사용자와 자연스럽게 상호작용하며 질문에 답변하거나 대화를 이끌어나가는 역할을 한다. 다양한 주제에 대해 대화가 가능하고, 실용적인 정보나 엔터테인먼트를 제공한다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 사용자는 다양한 필터와 효과를 이용하여 자신의 콘텐츠를 꾸밀 수 있고, 팔로워들과 소통할 수 있다. 또한 해시태그를 이용하여 관심사나 주제별로 콘텐츠를 찾을 수 있다.']

    - max_concurrency 매개변수를 사용해 동시 요청 수를 설정할 수 있음
    config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정함
    여기서는 최대 3개 작업ㅇ르 동시에 처리하도록 설정함

In [31]:
chain.batch(
    [
        {"topic" : "ChatGPT"},
        {"topic" : "Instagram"},
        {"topic" : "멀티모달"},
        {"topic" : "프로그래밍"},
        {"topic" : "머신러닝"},
    ],
    config = {"max_concurrency" : 3},
)

['ChatGPT는 인공지능 챗봇으로 자연어 처리 기술을 사용하여 대화를 수행한다. 사용자의 질문에 답변하거나 대화를 이어가며 다양한 주제에 대해 이야기를 나눈다. ChatGPT는 업데이트를 통해 계속 발전하고 새로운 기능과 지식을 습득하여 사용자에게 더 나은 대화 경험을 제공한다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 사용자들은 팔로워들과 소통하며 일상을 공유하고 취향에 맞는 콘텐츠를 발견할 수 있다. 해시태그를 통해 특정 주제나 관심사에 맞는 포스트를 찾을 수도 있다.',
 '멀티모달은 여러 가지 다양한 형태의 미디어를 결합하여 정보를 전달하거나 상호작용하는 것을 의미합니다. 이는 텍스트, 이미지, 비디오, 오디오 등 다양한 형식을 함께 사용하여 사용자 경험을 향상시키는 방법입니다. 멀티모달은 사용자가 자신에게 가장 편리하고 이해하기 쉬운 방식으로 정보를 받을 수 있도록 도와줍니다.',
 '프로그래밍은 컴퓨터에게 실행할 일련의 명령을 작성하는 과정이다. 이를 통해 우리는 컴퓨터가 원하는 작업을 수행하도록 할 수 있다. 프로그래밍은 문제 해결능력과 논리적 사고력을 기를 수 있는 유용한 기술이다.',
 '머신러닝은 컴퓨터 시스템이 데이터를 분석하고 학습하여 패턴을 발견하고 예측하는 인공지능 기술이다. 이를 통해 기계는 스스로 학습하고 결정을 내릴 수 있으며, 사람의 개입 없이 작업을 수행할 수 있다. 머신러닝은 이미지 인식, 자연어 처리, 음성 인식 등 다양한 분야에서 활발히 활용되고 있다.']

In [32]:
# 옵션 변경

chain.batch(
    [
        {"topic" : "ChatGPT"},
        {"topic" : "Instagram"},
        {"topic" : "멀티모달"},
        {"topic" : "프로그래밍"},
        {"topic" : "머신러닝"},
    ],
    config = {"max_concurrency" : 5},
)

['ChatGPT는 자연어 처리 기술을 이용하여 사용자와 대화하는 인공지능 챗봇 서비스이다. ChatGPT는 사용자의 질문에 답변하고 대화를 이어나가며 다양한 주제에 대해 대화할 수 있다. 사용자 경험을 개선하기 위해 지속적으로 학습하고 발전하는 시스템이다.',
 'Instagram은 사진과 동영상을 공유하는 SNS 플랫폼으로, 사용자들은 자신의 일상을 담은 콘텐츠를 업로드하고 다른 사람들과 소통할 수 있다. 해시태그를 통해 관심사나 주제별로 콘텐츠를 검색하고 팔로우하며, 인기 있는 게시물은 탐색하기 페이지에 노출된다. 또한 스토리 기능을 통해 일시적인 콘텐츠를 공유하고, 다양한 필터와 편집 기능을 활용하여 창의적이고 아름다운 사진을 만들 수 있다.',
 '멀티모달은 여러 가지 다른 형태의 정보를 함께 사용하여 효과적으로 의사소통하고 상호작용하는 것을 말합니다. 예를 들어 음성, 텍스트, 그림, 동영상 등 여러 가지 매체를 결합하여 정보를 전달하고 받는 방식이며, 시각적, 청각적, 운동적 등 다양한 감각을 활용하여 사용자 경험을 향상시킵니다. 멀티모달 접근법은 정보 전달의 효율성과 효과성을 높이는 데 도움을 줄 뿐만 아니라, 언어적이거나 문화적인 장벽을 극복하는 데도 유용하게 활용됩니다.',
 '프로그래밍은 컴퓨터에게 실행할 일련의 명령을 작성하는 과정이다. 이를 통해 우리는 복잡한 작업을 자동화하고 문제를 해결할 수 있다. 프로그래밍은 언어에 따라 다양한 방식으로 표현되며, 개발자는 코드를 작성하여 원하는 결과를 얻을 수 있다.',
 '머신러닝은 컴퓨터가 데이터를 이용해 패턴을 학습하고 예측하는 인공지능 기술이다. 이를 통해 컴퓨터는 데이터를 분석하고 의사결정을 내릴 수 있게 된다. 머신러닝은 이미지 인식, 자연어 처리, 예측 분석 등 다양한 분야에서 활용되고 있다.']

####  async stream : 비동기 스트림
    - chain.astream 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기로 처리
    - 비동기 for 루프(async for) 사용해 스트림에서 메시지를 순차적으로 받아오고, print 함수를 통해서 메시지의 내용(s.content)를 즉시 출력
    end = ""출력 후 줄바꿈을 하지 않도록 설정 flush=True 는 출력 버퍼를 강제로 비워 즉시 출력되도록 함

In [33]:
async for token in chain.astream({"topic" : "YouTube"}):
    print(token, end="", flush=True)

YouTube는 동영상을 올리고 시청할 수 있는 온라인 동영상 플랫폼이다. 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있으며, 구독자들과 소통하고 콘텐츠를 공유할 수 있다. 유명인들과 크리에이터들이 활발하게 활동하며, 광고 수익을 얻을 수 있는 플랫폼으로도 유명하다.

#### async invoke : 비동기 호출
    
    -   chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용해 작업을 수행함
    여기서 topic 이라는 키와 NVDA 라는 값을 가진 딕셔너리를 인자로 전달하고 있음
    이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는데 사용될 수 있음

In [37]:
chain.invoke({"topic" : "NVDA"})

'NVDA는 NVIDIA Corporation의 주식 기호이며, 미국의 반도체 기업으로 GPU(그래픽 처리 장치)를 주력 제품으로 생산한다. 최근 AI와 자율 주행차 기술 발전으로 주목을 받고 있으며, 기술적인 혁신을 이끄는 기업 중 하나이다. 주가는 기술 산업의 성장에 따라 변동성이 크며 투자자들 사이에서 인기를 끌고 있다.'

In [35]:
#비동기 체인 객체의 'ainvoke' 메서드로 'NVDA' 토픽 처리 

my_process = chain.ainvoke({"topic" : "NVDA"}) 
my_process

/var/folders/14/q73v_xlx3_x8kr742lgx5_sh0000gn/T/ipykernel_50882/3212333502.py:3: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  my_process = chain.ainvoke({"topic" : "NVDA"})


<coroutine object RunnableSequence.ainvoke at 0x110929540>

In [36]:
await my_process

'NVDA는 미국의 기업으로서 컴퓨터 그래픽스와 인공지능 기술을 전문으로 하는 회사이다. 주로 GPU와 시각 장애인을 위한 소프트웨어를 개발하며, 그래픽 카드 시장에서 선두적인 위치를 차지하고 있다. 혁신적인 기술과 제품으로 인해 글로벌 시장에서 큰 인정을 받고 있으며, 성장 가능성이 높은 기업으로 평가받고 있다.'

#### async batch: 비동기 배치

    -  함수 abatch는 비동기적으로 일련의 작업을 일괄 처리
    - await 키워드는 해당 비동기 작업이 완료될 때까지 기다리는데 사용

In [38]:
my_abatch_process = chain.abatch(
    [{"topic" : "YouTube"}, {"topic": "Instagram"}, {"topic" : "Facebook"}]
)


In [39]:
await my_abatch_process

['YouTube는 구글이 소유하고 있는 동영상 공유 플랫폼으로, 사용자들은 자신의 동영상을 업로드하고 시청할 수 있다. 다양한 콘텐츠를 제공하여 음악, 뉴스, 교육 등 다양한 주제의 동영상을 즐길 수 있다. 또한 라이브 스트리밍 기능을 통해 실시간으로 이벤트나 방송을 시청할 수도 있다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크를 형성할 수 있는 앱이다. 사용자들은 팔로워들과 소통하며 일상을 공유하고 콘텐츠를 발견할 수 있다. 해시태그와 인플루언서 마케팅을 통해 다양한 사용자들과 연결되는 플랫폼이다.',
 'Facebook은 세계적으로 가장 대중적인 소셜 네트워크 서비스 중 하나로, 사람들이 친구들과 소통하고 정보를 공유하는 플랫폼이다. 사용자들은 텍스트, 사진, 동영상 등 다양한 콘텐츠를 게시하고 다른 사람들과 소통할 수 있다. 또한 광고를 통해 비즈니스 활동을 홍보하고 마케팅을 할 수 있는 기회를 제공한다.']

### Parallel : 병렬성
    - RunnableParallel을 사용할 때, 각 요소를 병렬로 실행함
    사용자 정의 체인을 최대한 쉽게 만들 수 있도록, "Runnable" 프로토콜 구현
    

In [42]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model 
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

combined = RunnableParallel(captical=chain1, area=chain2)


In [43]:
chain1.invoke({"country" : "대한민국"})

'대한민국의 수도는 서울이다.'

In [44]:
chain2.invoke({"country":"대한민국"})

'대한민국의 면적은 약 100,363.4 km² 입니다.'

In [45]:
combined.invoke({"country": "대한민국"})

{'captical': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}

In [46]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country1}의 수도는 어디야?")
    | model 
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country2}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

combined = RunnableParallel(captical=chain1, area=chain2)

In [47]:
combined.invoke({"country1" : "대한민국", "country2" : "대한민국"})

{'captical': '대한민국의 수도는 서울이야.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}

In [48]:
combined.invoke({"country1" : "대한민국", "country2" : "미국"})

{'captical': '대한민국의 수도는 서울이야.', 'area': '미국의 총 면적은 대략 9,833,520km² 입니다.'}

In [49]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model 
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

combined = RunnableParallel(captical=chain1, area=chain2)

In [50]:
# 배치처리

chain1.batch([{"country" : "대한민국"}, {"country" : "캐나다"}])

['대한민국의 수도는 서울이야.', '캐나다의 수도는 오타와(Ottawa)야.']

In [51]:
chain2.batch([{"country" : "대한민국"}, {"country":"캐나다"}])

['대한민국의 총 면적은 약 100,363㎢ 입니다.', '캐나다의 면적은 약 9,984,670 제곱 킬로미터입니다.']